In [1]:
import numpy as np
import pandas as pd
from queue import Queue
import matplotlib.pyplot as plt
import matplotlib.path as mpath
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
from io import StringIO

In [ ]:
class AEMinimo(object):
  def __init__(self, mat, umbral):
    self.__mat = mat
    self.__n = len(mat)
    self.adj = [[] for i in range(self.__n)]
    self.__umbral = umbral
    self.costoTotal = 0
    self.__prim()

  def __prim(self):
    self.costoTotal = 0
    pesos = [float("inf")] * self.__n
    usados = [False] * self.__n
    desde = [0] * self.__n
    pesos[0] = 0
    nodo = 0
    for i in range(1, self.__n):
      usados[nodo] = True
      index = -1
      pesoMin = float("inf")
      for j in range(self.__n):
        if self.__mat[nodo][j] > self.__umbral and self.__mat[nodo][j] < pesos[j] and usados[j] == False:
          pesos[j] = self.__mat[nodo][j]
          desde[j] = nodo
        if usados[j] == False and pesos[j] < pesoMin:
          index = j
          pesoMin = pesos[j]
      if index == -1:
        for j in range(self.__n):
          if usados[j] == False:
            pesos[j] = 0
            nodo = j
            break
      else:
        self.__agregarArista(desde[index], index)
        nodo = index
        self.costoTotal += pesoMin

  def __agregarArista(self, a, b):
    self.adj[a].append(b)
    self.adj[b].append(a)



In [ ]:
class AEMaximo(object):
  def __init__(self, mat, umbral):
    self.__mat = mat
    self.__n = len(mat)
    self.adj = [[] for i in range(self.__n)]
    self.__umbral = umbral
    self.costoTotal = 0
    self.__prim()

  def __prim(self):
    self.costoTotal = 0
    pesos = [float("-inf")] * self.__n
    usados = [False] * self.__n
    desde = [0] * self.__n
    pesos[0] = 0
    nodo = 0
    for i in range(1, self.__n):
      usados[nodo] = True
      index = -1
      pesoMax = float("-inf")
      for j in range(self.__n):
        if self.__mat[nodo][j] > self.__umbral and self.__mat[nodo][j] > pesos[j] and usados[j] == False:
          pesos[j] = self.__mat[nodo][j]
          desde[j] = nodo
        if usados[j] == False and pesos[j] > pesoMax:
          index = j
          pesoMax = pesos[j]
      if index == -1:
        for j in range(self.__n):
          if usados[j] == False:
            pesos[j] = 0
            nodo = j
            break
      else:
        self.__agregarArista(desde[index], index)
        nodo = index
        self.costoTotal += pesoMax

  def __agregarArista(self, a, b):
    self.adj[a].append(b)
    self.adj[b].append(a)


In [ ]:
class ModeloAE(object):
  def __init__(self, archivo, tipo):
    self.__tipo = tipo
    self.__archivo = archivo
    self.__raices = []
    self.__leerDatos(self.__archivo)
    self.__n = len(self.__datos)
    self.__tamanos = [0 for i in range(self.__n)]
    self.__adjDirigida = [[] for i in range(self.__n)]
    self.__nombres = [str(i) for i in range(self.__n)]
    self.__redondear()
    self.__arbol = None

  def __leerDatos(self, archivo):
    data = np.genfromtxt(archivo, delimiter=',')
    filas = len(data)
    if filas != len(data[0]):
      self.__convertirAMatriz(data)
    else:
      self.__datos = [[data[i][j] for i in range(filas)]for j in range(filas)]

  def __convertirAMatriz(self, lista):
    vertices = 0
    for w, a, b in lista:
      vertices = max(vertices, a, b)
    vertices = int(vertices) + 1
    self.__datos = [[0 for i in range(vertices)] for j in range(vertices)]
    for w, a, b in lista:
      self.__datos[int(a)][int(b)] = float(w)
      self.__datos[int(b)][int(a)] = float(w)


  def leerNombres(self, archivo):
    csv = open(archivo, "r")
    for i in range(self.__n):
      self.__nombres[i] = csv.readline()
      
  def __redondear(self):
    for i in range(self.__n):
      for j in range(self.__n):
        x = self.__datos[i][j] * 10
        self.__datos[i][j] = round(x)

  def guardarCSV(self, nombre):
    pd.DataFrame(self.__datos).to_csv(nombre+'.csv', header=None, index=None)

  def conseguirAEM(self, u=0):
    self.__raices = []
    self.__tamanos = [0 for i in range(self.__n)]
    self.__adjDirigida = [[] for i in range(self.__n)]
    if self.__tipo == "min":
      self.__arbol = AEMinimo(self.__datos, u)
    else:
      self.__arbol = AEMaximo(self.__datos, u)
    self.__conseguirRaices()

  def __conseguirRaices(self):
    self.__raices = []
    usados = [False] * self.__n
    for i in range(self.__n):
      if usados[i] == False:
        usados[i] = True
        self.__raices.append(self.__conseguirRaiz(i, usados))

  def __conseguirRaiz(self, index, usados):
    arr, tam = self.__conseguirComponente(index)
    grado = [len(self.__arbol.adj[i]) for i in range(self.__n)]
    cola = Queue(maxsize = self.__n)
    for x in arr:
      cola.put(x)
    while cola.qsize() > 1:
      nodo = cola.get()
      usados[nodo] = True
      grado[nodo] = -1
      for x in self.__arbol.adj[nodo]:
        grado[x] -= 1
        if grado[x] >= 0:
          self.__adjDirigida[x].append(nodo)
        if grado[x] == 1:
          cola.put(x)
    raiz = cola.get()
    usados[raiz] = True
    self.__tamanos[raiz] = tam
    return raiz

  def __conseguirComponente(self, index):
    usados = [False] * self.__n
    usados[index] = True
    cola = Queue(maxsize=self.__n)
    cola.put(index)
    tam = 0
    ans = []
    while cola.qsize() > 0:
      nodo = cola.get()
      tam += 1
      if len(self.__arbol.adj[nodo]) <= 1:
        ans.append(nodo)
      for x in self.__arbol.adj[nodo]:
        if usados[x] == False:
          usados[x] = True
          cola.put(x)
    return (ans, tam)

  def verResultados(self):
    print("Se encontraron {} árboles de expansión con un peso total de {}.".format(len(self.__raices), self.__arbol.costoTotal))
    print("La lista de adyacencia de los árboles encontrados es:")
    for linea in self.__arbol.adj:
      print(linea)

  def verEnCuadricula(self):
    self.__verArboles(1)

  def verRelativo(self):
    self.__verArboles(2)

  def __verArboles(self, forma):
    for i in range(len(self.__raices)):
      global y
      y = 10
      fig, ax = plt.subplots(figsize=(20,20))
      plt.title('Componente #{}'.format(i+1))
      plt.axis('off')
      if forma == 1:
        maxY = self.__contarVertices(self.__raices[i])
        h = (maxY - 1) * 20
        plt.xlim([0,h+20])
        plt.ylim([h+20,0])
      else:
        maxY = self.__contarVertices(self.__raices[i])
        h = (maxY - 1) * 20
        maxX = self.__calcularLongitudMaxima(self.__raices[i])
        plt.xlim([0,max(h, maxX)+20])
        plt.ylim([max(h, maxX)+20,0])
      self.__f1(fig, ax, 10, self.__raices[i], 10, 10, forma, 0)


  def __calcularLongitudMaxima(self, raiz):
    ans = 0
    for i in self.__adjDirigida[raiz]:
      ans = max(ans, self.__conseguirLongitud(2, self.__datos[raiz][i]) + self.__calcularLongitudMaxima(i))
    return ans

  def __contarVertices(self, raiz):
    ans = 0
    for i in self.__adjDirigida[raiz]:
      ans += self.__contarVertices(i)
    return ans + 1
  
  def __f1(self, fig, ax, x, index, prevx, prevy, forma, pesoAnterior):
    global y
    tmp = y
    c = plt.Circle((x, y), 5, color='r')
    l1 = (x, prevx)
    l2 = (y, prevy)
    plt.plot([prevx, prevx], [prevy, y], 'k-', lw=2)
    plt.plot([prevx, x], [y, y], 'k-', lw=2)
    ax.add_patch(c)
    plt.text(x+4, y, self.__nombres[index], fontsize=8)
    if pesoAnterior != 0:
      plt.text(prevx+2, y+12, str(pesoAnterior), fontsize=8)
    for i in self.__adjDirigida[index]:
      y += 20
      aumentoX = self.__conseguirLongitud(forma, int(self.__datos[index][i]))
      self.__f1(fig, ax, x+aumentoX, i, x, tmp, forma, int(self.__datos[index][i]))

  def __conseguirLongitud(self, forma, peso):
    if forma == 1:
      return 20
    else:
      if self.__tipo == "min":
        return (peso * 7)
      else:
        return ((11 - peso) * 7)

  def verCirculo(self):
    global indexPos
    indexPos = 0
    indexColores = 0
    colores = ['b','g','c','r','m','y']
    fig, ax = plt.subplots(figsize=(20,20))
    plt.title("Resultado")
    plt.axis('off')
    plt.xlim([-1050,1050])
    plt.ylim([-1050,1050])
    pos = self.__posicionesCirculo(980, self.__n)
    for r in self.__raices:
      self.__f2(fig, ax, pos[indexPos][0], pos[indexPos][1], r, pos, self.__adjDirigida, colores[indexColores % 6])
      indexPos += 1
      indexColores += 1

  def __posicionesCirculo(self, radio, n):
    angulos = np.linspace(0, 2*np.pi, n, endpoint=False)
    x = radio * np.sin(angulos)
    y = radio * np.cos(angulos)
    ans = []
    for i in range(n):
      ans.append((x[i], y[i]))
    return ans

  def __f2(self, fig, ax, prevx, prevy, indexAdj, pos, adj, color):
    global indexPos
    ax.scatter(pos[indexPos][0], pos[indexPos][1], c=color, s = 100)
    if indexPos < (len(pos) // 2):
      xEtiqueta = pos[indexPos][0] + 15
    else:
      xEtiqueta = pos[indexPos][0] - 40
    if indexPos < (len(pos) // 4) or indexPos >= (3 * len(pos) // 4):
      yEtiqueta = pos[indexPos][1] + 15
    else:
      yEtiqueta = pos[indexPos][1] - 30
    plt.text(xEtiqueta, yEtiqueta, str(indexAdj), fontsize=8)
    for x in adj[indexAdj]:
      indexPos += 1
      plt.plot([prevx, pos[indexPos][0]], [prevy, pos[indexPos][1]], 'k-', lw=1.0)
      self.__f2(fig, ax, pos[indexPos][0], pos[indexPos][1], x, pos, adj, color)

    